In [102]:
import seaborn as sns
import pandas as pd
import numpy as np

In [103]:
df = pd.read_json('data/data.json')
df.head()
df.describe()
df.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state'],
      dtype='object')

In [104]:
df['user_type'].unique()
df['acct_type'].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

# Step 1: EDA

In [105]:
#1
df['fraud'] = df['acct_type'].apply(lambda x: 'fraud' in x)
df['fraud']

0         True
1        False
2        False
3        False
4        False
         ...  
14332     True
14333    False
14334    False
14335    False
14336     True
Name: fraud, Length: 14337, dtype: bool

In [106]:
df.drop(columns=['acct_type'], inplace=True)

In [107]:
#2
df[df['fraud']==True]['fraud'].count()
df[df['fraud']==False]['fraud'].count()
# Fraud = ~10%

13044

In [108]:
#3
features = ['body_length'    ,
'channels'       ,
'delivery_method',
'has_logo'       ,
'name_length'    ,
'org_facebook'   ,
'org_twitter'    ,
'sale_duration'  ,
'sale_duration2' ,
'user_age'       ,
'fraud']


In [109]:
df = df[features]

### Scoping the problem

1.) Preprocessing
- Convert dates to dt format
- Convert binary categories to boolean

2.) Models to Try

3.) Metric to use

# Step 2: Building the Model

In [110]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df.dropna(inplace=True)


X = df.drop(columns=['fraud'])
y = df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [114]:
X_train.dtypes

body_length          int64
channels             int64
delivery_method    float64
has_logo             int64
name_length          int64
org_facebook       float64
org_twitter        float64
sale_duration      float64
sale_duration2       int64
user_age             int64
dtype: object

In [125]:
def logistic_model(X, y):
    from sklearn.linear_model import LogisticRegression
    
    #select the columns with numerical/categorical data
    #X = X.loc[:, (X.dtypes == np.float64) or (X.dtypes == np.int64)]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    
   
    
    #fit model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    coeffs = pd.DataFrame(model.coef_, columns=X_train.columns)
    print(coeffs)
    
    return model, accuracy


def SGD_model(X, y):
    from sklearn.linear_model import SGDClassifier
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y)
    
    #fit model
    model = SGDClassifier()
    model.fit(X_train, y_train)
    accuracy = model.score(X_valid, y_valid)
    model.
    
    return model, accuracy
    
    

In [126]:
logistic_model(X_train, y_train)
SGD_model(X_train, y_train)

   body_length  channels  delivery_method  has_logo  name_length  \
0    -0.000064 -0.063318        -0.055506 -0.017621    -0.000287   

   org_facebook  org_twitter  sale_duration  sale_duration2  user_age  
0     -0.067618    -0.135244       0.046525       -0.073409 -0.002907  


/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(SGDClassifier(alpha=0.0001, average=False, class_weight=None,
               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
               l1_ratio=0.15, learning_rate='optimal', loss='hinge',
               max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
               power_t=0.5, random_state=None, shuffle=True, tol=0.001,
               validation_fraction=0.1, verbose=0, warm_start=False),
 0.9166037735849056)